In [74]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss
import tensorly as tl
from tensorly.regression.tucker_regression import TuckerRegressor
from jax import grad
from references.SABR_Free import sabrMC

try:
    tl.set_backend('jax')
except:
    pass
tl.set_backend('jax')

# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

In [62]:
sample=''
sample = np.genfromtxt('references/sample.csv', delimiter=',')

In [34]:
# check distribution
print(sample.shape) # paths, time
# print(S[:,-1][:5])

# print(np.mean(sample.T[:,-1]))
# print(np.std(sample.T[:,-1]))

print(np.mean(sample.T[:,-1]) * 1.05 - 1.05)
print(np.std(sample.T[:,-1]) * (1.05**2) - 0.26667)

# 1.05
# 0.2667; =sqrt(1.05^2(e^(0.25^2*1)-1))

(100, 10000)
0.001832487937137861
0.011829146522039147


In [77]:
def put_price(r=0.05, paths = 10000, steps=100, K=1.0, maturity = 1.0, data=None, seed=1):

    if data is not None:
        S=data * (1+r)
        print(type(S))
    else:
        np.random.seed(seed)
        S = sabrMC(F0 = 1+r, N=paths, T=steps, n_years=1).T
        
    dt = maturity/(steps-1)         # time interval
    df = jnp.exp(-r * dt)           # discount factor per time interval
    
    H = jnp.maximum(K - S, 0)           # intrinsic values for put option
    V = jnp.zeros(H.shape)               # value matrix
    V = V.at[:,-1].set(H[:,-1]) 

    # Valuation by LS Method
    for t in range(steps-2, 0, -1):

        good_paths = np.array(H[:,t] > 0)        # paths where the intrinsic value is positive 
                                                 # the regression is performed only on these paths 

        # Valuation by Tucker Regression
        X = S[good_paths, t].reshape((-1,1,1)) # reshape to fill up dimensions
        y = V[good_paths, t+1] * df
        estimator = TuckerRegressor(weight_ranks=[1]*X.ndim, verbose=False, random_state=seed) # hyperparameter: reg_W (default: 1) 
        estimator.fit(X, y)
        fitted_values = estimator.predict(X) # predicted value of option (discounted, as `y` is discounted)

        exercise = jnp.zeros(len(good_paths)).astype(bool) # initialize
        # early exercise when value of early exericse at t > predicted value at t+1 (discounted to t)
        exerise = exercise.at[good_paths].set(H[good_paths,t] > fitted_values)

        V = V.at[exercise,t].set(H[exercise,t]) # set V equal to H where it is optimal to exercise 
        V = V.at[exercise,t+1:].set(0) # set future cash flows, for that path, equal to zero  
        discount_path = V[:,t] == 0 # paths where we didn't exercise
        V = V.at[discount_path,t].set(V[discount_path,t+1] * df) # set V[t] in continuation region

    V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
    return V0

print("Example price= ", put_price(data=sample.T)) # should be 0.0797

<class 'numpy.ndarray'>
Example price=  0.07357290525046334


In [79]:
print('delta:')
delta = grad(put_price, argnums=0)
print(delta(0.05, paths = 100, steps=100, K=1.0, maturity = 1.0, data=sample.T))
print('gamma:')
print(grad(delta)(0.05, paths = 100, steps=100, K=1.0, maturity = 1.0, data=sample.T))



delta:
<class 'jax.interpreters.ad.JVPTracer'>
-0.42005136633138007
gamma:
<class 'jax.interpreters.ad.JVPTracer'>
0.7665298274122965
